In [1]:
# PLEASE NOTE:
# This software was mostly made for convenient use with command-line interface (CLI),
# but sortasurvey.notebook contains copies of similar classes and functions to be easier 
# implemented in scripts or jupyter notebooks for demonstrative purposes

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sortasurvey.notebook import Survey, rank

### The software is centered around the Survey class

In [2]:
# The software is centered around the Survey class, which contains both programmatic
# information (via path_survey) and the sample to select from (via path_sample) for
# the survey. There are options for any high priority targets and similarly, targets
# to ignore in the selection process. The the latter two must have columns that match
# the program names (str) within the surveyl (default is `None`)

path_sample='../info/TOIs_perfect.csv'
path_survey='../info/survey_info.csv'
path_priority='../info/high_priority.csv'
path_ignore='../info/no_no.csv'

TKS = Survey(path_sample=path_sample, path_survey=path_survey, path_priority=path_priority, path_ignore=path_ignore)
# As expected, the sample is stored to the Survey.sample attribute
candidates = TKS.sample
# We can take a look at what columns are available for target selection purposes
print(candidates.columns.values.tolist())

['toi', 'tic', 'cps_name', 'evol', 'source', 'disp', 'photo_vetting', 'spec_vetting', 'ao_vetting', 'ra', 'dec', 'tmag', 'tmag_err', 'bmag', 'vmag', 'b_v', 'jmag', 'hmag', 'kmag', 'gmag', 'nobs', 'last_observed', 'template', 'nobs_goal', 'tot_time', 'rem_nobs', 'rem_time', 'rs', 'rs_err', 'r_s', 'm_s', 'teff', 'teff_err', 't_eff', 'teff_jump', 'logg', 'logg_err', 'logg_jump', 'feh', 'f_eh', 'ast_det_c', 'c_sector', 'ast_det_t', 't_sector', 'color', 'vsini', 'prot', 'sval', 'sigma_act', 'sigma_rot', 'sigma_gran', 'sigma_rv', 'sigma_tot', 'limit', 'logrhk', 'ruwe', 'rp', 'rp_err', 'mp', 'k', 'kerr', 'ksig', 'epoch', 'epoch_err', 'period', 'period_err', 'dur', 'dur_err', 'a_to_R', 'depth', 'depth_err', 'teq', 'sinc', 'SNR', 'nobs_5sig', 'rt_5sig', 'tt_5sig', 'nobs_10sig', 'rt_10sig', 'tt_10sig', 'alerted', 'updated', 'drop', 'finish', 'npl', 'select_DG', 'in_other_programs', 'n_select', 'in_SC1A', 'in_SC1B', 'in_SC1C', 'in_SC1D', 'in_SC1E', 'in_SC2A', 'in_SC2Bi', 'in_SC2Bii', 'in_SC2C', '

### View TKS science programs

In [3]:
# Similarly, all TKS science-case specific information is stored to Survey.programs,
# a pandas.DataFrame with program names as the index to make it easier to access 
# specific information

# Here is the complete list of TKS programs
print(TKS.programs.index.values.tolist())

['SC1A', 'SC1B', 'SC1C', 'SC1D', 'SC1E', 'SC2A', 'SC2Bi', 'SC2Bii', 'SC2C', 'SC3', 'SC4', 'TOA', 'TOB']


### Get specific details for a specific TKS program

In [4]:
# Let's take a look at the TKS multis program
case = 'SC2C'

print(TKS.programs.loc[case])

name                                                            SC2C
method                                    hires-nobs=100-counts=ramp
filter             photo_vetting == 'passed' and spec_vetting != ...
prioritize_by                                     [npl, actual_cost]
ascending_by                                           [False, True]
remaining_hours                                               47.619
n_maximum                                                          4
total_time                                                    47.619
high_priority                                                     []
n_targets_left                                                     4
pick_number                                                        0
Name: SC2C, dtype: object


In [5]:
# Primarily, each science case within a survey needs:
# 1) an observing strategy (aka method)
# 2) selection criteria (aka filter using the sample columns)
# 3) prioritization metrics (can be more than one, as is the case here)
# 4) allocated time
# and optionally:
# 5) a set number of selection (n_maximum, which is 4 here. so SC2C can only select up to 4 targets.)
# 
# The values for 'remaining_hours' and 'pick_number' will be updated during
# the selection process, providing ways to identify when the selection process
# would crash, i.e. if a selected program cannot afford their selected target
# and/or if the program runs out of selections

## $\rm Run\; ranking\; algorithm$

In [6]:
rank(TKS)


 -- prioritization starting --

   - loading sample and survey science information
   - 785 targets make the standard survey cuts
   - 242 have also passed various vetting steps
   - ranking algorithm initialized
   - algorithm took 44 seconds to run
   - 83 targets were selected
   - making data products
   - copy of updated sample information saved
   - algorithm history saved as ranking steps
   - final prioritized list saved
   - program overlap csv saved
   - txt file containing run information saved

 --- process complete ---

Tue 06/08/21 04:59PM
Seed no: 2222
Out of the 83 total targets:
  - SC1A has 27 targets
  - SC1B has 19 targets
  - SC1C has 6 targets
  - SC1D has 4 targets
  - SC1E has 3 targets
  - SC2A has 40 targets
  - SC2Bi has 5 targets
  - SC2Bii has 0 targets
  - SC2C has 4 targets
  - SC3 has 20 targets
  - SC4 has 16 targets
  - TOA has 83 targets
  - TOB has 8 targets



---------------------------------------------------------------------------------------------------------------------

# $\rm \textbf{SUMMARY RESULTS} $

### $\rm Number\; of\; targets\; in\; each\; program$

In [ ]:
data.get_stats()

### $\rm See\; selected \; targets$

In [ ]:
print(data.observed)

### $\rm See\; overlap$

In [ ]:
print(data.overlap.head())

## $\rm Check\; amount\; of\; overlap\; for\; targets \; selected\; from\; a\; given\; program$

In [ ]:
science_case = 'SC2A'

program = data.overlap.query("in_%s == 'X'"%science_case)

# how many have 5 or more programs?
query = program.query('total_programs >= 5')
print('%d picked targets had large overlap (>= 5 programs) with other programs'%len(query))

## $\rm Knowing\; the\; starting\; sample\; vs\; the \; final\; sample\; for\; a\; given\; program$

In [ ]:
from ranking import Sample

science = 'SC2A'
tks = Sample(data.final)

sc2 = tks.in_science(science)
print('There was %d relevant targets to choose from'%len(sc2))
sc2 = tks.in_sample(science)
print('%d of them were selected'%len(sc2))

In [ ]:
%pylab inline


df2 = data.final
program = data.final.query("in_%s == 1"%science_case)

rp_all = df2.rp.values.tolist()
sinc_all = df2.sinc.values.tolist()
rp = program.rp.values.tolist()
sinc = program.sinc.values.tolist()

plt.figure(figsize = (14,8))

ax = plt.subplot(1,1,1)
p1 = (1e6, 2.1)
p2 = (800, 3.8)
plt.plot((p1[0], p2[0], p2[0], p1[0]), (p1[1], p1[1], p2[1], p2[1]), 'k--', zorder = 0, label = r'$\rm Super$-$\rm Earth \; Desert$')
ax.scatter(sinc_all, rp_all, c = '0.75', s = 100., zorder = 0, label = r'$\rm Full \; Sample \; [%s]$'%(str(int(len(rp_all)))))
ax.scatter(sinc, rp, c = '#ff8c00', s = 100., lw = 0.5, edgecolor = 'b', zorder = 2, label = r'$\rm %s \; [%s]$'%(science, str(int(len(rp)))))
ax.set_ylabel(r"$\rm R_{p}\,\, [R_{\oplus}]$", fontsize = 24.)
ax.set_xlabel(r"$\rm Incident\,\, Flux\,\, [F_{\oplus}]$", fontsize = 24.)
ax.tick_params(axis = 'both', which = 'minor', length = 5, width = 1, direction = 'inout')
ax.tick_params(axis = 'both', which = 'major', length = 8, width = 2, direction = 'inout')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')
plt.yscale("log")
plt.xscale("log")
ax.axvspan(0.25, 10., facecolor = 'g', alpha = 0.25, zorder = 0, label = r'$\rm Optimistic \; HZ$')
ax.axvspan(0.25, 1.1, facecolor = 'g', alpha = 0.75, zorder = 0, label = r'$\rm Conservative \; HZ$')
ax.set_xticks([0.1, 1, 10, 100, 1000, 10000, 100000])
ax.set_xticklabels([r'$10^{-1}$', r'$10^{0}$', r'$10^{1}$', r'$10^{2}$', r'$10^{3}$', r'$10^{4}$', r'$10^{5}$'])
ax.set_yticks([1, 10])
ax.set_yticklabels([r'$1$', r'$10$'])
ax.legend(fontsize = 20., loc = 'center left', bbox_to_anchor = (1, 0.5), markerscale = 1.25)
plt.tick_params(labelsize = 18)
plt.xlim([15000., 1.])
plt.ylim([0.9, 22.])
plt.tight_layout()